In [1]:
%matplotlib inline
import pandas as pd
import sklearn
import numpy as np
from IPython.display import display
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from matplotlib import rcParams

In [2]:
import pickle
import codecs
import collections
import keras.preprocessing
from sklearn.model_selection import KFold
import csv

Using TensorFlow backend.


In [4]:
from keras.models import Sequential, Model
from keras.layers import Embedding, Reshape, Activation, Input, Dense,GRU,Reshape,TimeDistributed,Bidirectional,Dropout,Masking
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.layers import Embedding,TimeDistributed,Flatten
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import GRU, Dropout
from keras.models import load_model
from keras.layers import Embedding, Reshape, Activation, Input, Dense, Masking
from keras.layers.merge import Dot
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.utils.np_utils import to_categorical
from keras.preprocessing.sequence import skipgrams
from keras.preprocessing import sequence
from keras import backend as K
from keras.optimizers import Adam
from keras.layers.convolutional import Conv1D
from keras.layers import Embedding,TimeDistributed,Flatten
from keras.layers import BatchNormalization,Dropout


In [5]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

In [6]:
data = [[-1, 2], [-0.5, 6], [0, 10], [1, 18]]
scaler = MinMaxScaler()
print(scaler.fit(data))

MinMaxScaler(copy=True, feature_range=(0, 1))


In [12]:
input_pbs = pd.read_csv('../data/Thaipbs-tokenize.csv')

In [13]:
input_pbs.drop('Unnamed: 0', axis=1, inplace=True)
# input_pbs.drop('Unnamed: 0.1', axis=1,inplace=True)
# input_pbs.drop('Unnamed: 1', axis=1, inplace=True)

In [14]:
input_pbs.head()

,headline,date,DOW,time,view,category,tag,hour,numTag,token,numToken,numChar
0,ฝุ่น PM2.5 : ทส.เตรียมตั้งศูนย์แก้ปัญหาหมอกควั...,05/04/2562,FRI,19:37,177,สิ่งแวดล้อม,"ฝุ่นเชียงใหม่,ฝุ่นPM2.5,ฝุ่นคลุมเมือง,เชียงใหม...",19,10,"ฝุ่น,PM,ทส.,เตรียม,ศูนย์,แก้,ปัญหา,หมอก,ควัน,ข...",11,43
1,"เลือกตั้ง 2562 : ""เพื่อไทย"" เตรียมยื่น กกต.จัด...",05/04/2562,FRI,19:20,702,การเมือง,"เลือกตั้ง62,เพื่อ่ไทย,กกต.,นับคะแนน,ไทยพีบีเอส...",19,6,"เลือกตั้ง,2562,ไทย,เตรียม,ยื่น,กกต.,เลือกตั้ง,...",9,46
2,เตรียมเอาผิด รพ.เอกชน 58 แห่งไม่ส่งข้อมูลราคายา,05/04/2562,FRI,19:18,583,สาธารณสุข,"ยา,เวชภัณฑ์ฯ,โรงพยาบาล,ค่าบริการทางการแพทย์,กร...",19,9,"เตรียม,รพ.เอกชน,ข้อมูล,ราคา,ยา",5,26
3,ตั้งข้อสังเกตปลดล็อกกัญชา เอื้อนายทุนหรือไม่?,05/04/2562,FRI,19:16,928,อาชญากรรม,"กัญชา,มูลนิธิข้าวขวัญ,ThaiPBSnews",19,3,"ข้อ,สังเกต,ปลด,ล็อก,กัญชา,เอื้อ,ทุน",7,29
4,"คำพิพากษาเต็ม ""เปรมชัย"" หลุดคดีครอบครองซากเสือดำ",05/04/2562,FRI,19:29,5163,สิ่งแวดล้อม,"เสือดำ,เปรมชัย,เขตรักษาพันธุ์สัตว์ป่าทุ่งใหญ่น...",19,5,"พิพากษา,เต็ม,เปรมชัย,หลุดคดี,ครอบครอง,ซาก,เสือดำ",7,42


In [15]:
IQRval = input_pbs['view'].describe().loc['75%']-input_pbs['view'].describe().loc['25%']

In [16]:
input_pbs['view'].describe().loc['25%']

252.0

In [17]:
outlierMin = max(input_pbs['view'].describe().loc['25%'],0)
outlierMax = input_pbs['view'].describe().loc['75%']+1.5*IQRval
print(outlierMin,outlierMax)

252.0 3499.5


In [18]:
input_pbs = input_pbs[(input_pbs['view']<=outlierMax) & (input_pbs['view']>=outlierMin)]

In [19]:
input_pbs['numToken'].describe()

count    17521.000000
mean         8.386565
std          2.509898
min          1.000000
25%          7.000000
50%          8.000000
75%         10.000000
max         20.000000
Name: numToken, dtype: float64

In [20]:
input_pbs['view'].describe()

count    17521.000000
mean       994.658924
std        772.189473
min        252.000000
25%        411.000000
50%        697.000000
75%       1337.000000
max       3496.000000
Name: view, dtype: float64

In [21]:
MAXVIEW = input_pbs['view'].max()
MAXVIEW

3496

## Feature extraction

In [22]:
input_pbs_train, input_pbs_test = train_test_split(input_pbs, test_size=0.2,random_state=42)

## Bag of word feature

In [23]:
inputText_train = []
for sent in input_pbs_train['token']:
    inputText_train.append(sent.split(','))

In [24]:
inputText_test = []
for sent in input_pbs_test['token']:
    inputText_test.append(sent.split(','))

In [25]:
inputLabel_train = []
for view in input_pbs_train['view']:
    inputLabel_train.append(view/MAXVIEW)

In [26]:
inputLabel_test = []
for view in input_pbs_test['view']:
    inputLabel_test.append(view/MAXVIEW)

In [27]:
print(len(inputText_train),len(inputLabel_train))

14016 14016


In [28]:
print(len(inputText_test),len(inputLabel_test))

3505 3505


In [386]:
words = []
for sentence in inputText_train:
    for word in sentence:
        words.append(word)
        
word_count = list()
word_count.extend(collections.Counter(words).most_common(len(set(words))))
word_count = word_count[:len(word_count)//4]
word_count.append(("UNK",0))

train_word = set()
for i in word_count:
    train_word.add(i[0])

In [387]:
len(word_count)

3514

In [388]:
all_token = set()
for word in train_word:
    all_token.add(word)

In [389]:
all_token = list(all_token)
all_token.insert(0,'for padding')
all_token.insert(1,'UNK')

In [390]:
len(all_token)

3516

In [391]:
token_map = dict(zip(all_token, range(len(all_token))))
token_map_reverse = dict(zip(range(len(all_token)),all_token))

In [392]:
input_data_train = np.asarray(input_pbs_train['token'].str.split(','))
input_data_test = np.asarray(input_pbs_test['token'].str.split(','))

In [393]:
def count_word(sent):
    global all_token, token_map
    result = np.zeros(len(all_token))
    np_token = np.array(sent)
    str_token, str_token_count = np.unique(np_token, return_counts=True)
    for char, count in zip(str_token, str_token_count):
        if char not in token_map.keys():
            char = 'UNK'
        result[token_map[char]] = count
    return result

In [394]:
temp_train = np.vectorize(count_word, otypes=[object])(input_data_train)
x_f1_train = np.array([[e for e in sl] for sl in temp_train.tolist()])
temp_test = np.vectorize(count_word, otypes=[object])(input_data_test)
x_f1_test = np.array([[e for e in sl] for sl in temp_test.tolist()])

## Numerical feature

In [29]:
cs = MinMaxScaler()

In [30]:
trainContinuous = cs.fit_transform(input_pbs_train[['numToken','numChar','numTag']])
testContinuous = cs.fit_transform(input_pbs_test[['numToken','numChar','numTag']])

/home/burin/py_36_env/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/burin/py_36_env/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


## Cetagorial feature

In [31]:
lb = preprocessing.LabelBinarizer()

In [32]:
zipBinarizer = lb.fit(input_pbs["DOW"])
trainCategorical_dow = zipBinarizer.transform(input_pbs_train["DOW"])
testCategorical_dow = zipBinarizer.transform(input_pbs_test["DOW"])
zipBinarizer = lb.fit(input_pbs["hour"])
trainCategorical_hour = zipBinarizer.transform(input_pbs_train["hour"])
testCategorical_hour = zipBinarizer.transform(input_pbs_test["hour"])
zipBinarizer = lb.fit(input_pbs["category"])
trainCategorical_category = zipBinarizer.transform(input_pbs_train["category"])
testCategorical_category = zipBinarizer.transform(input_pbs_test["category"])

## Concat feature

In [36]:
x_train = np.hstack([ trainContinuous,trainCategorical_dow,trainCategorical_hour,trainCategorical_category])
x_test = np.hstack([ testContinuous,testCategorical_dow,testCategorical_hour,testCategorical_category])

In [37]:
# x_train = np.hstack([x_f1_train,trainCategorical_dow,trainCategorical_hour,trainCategorical_category])
# x_test = np.hstack([x_f1_test,testCategorical_dow,testCategorical_hour,testCategorical_category])

In [38]:
# x_train = np.hstack([x_f1_train, trainContinuous])
# x_test = np.hstack([x_f1_test, testContinuous])

## Prepare data for Keras

In [39]:
x_train=keras.preprocessing.sequence.pad_sequences(x_train, maxlen=None, dtype='int32', padding='post', truncating='pre', value=0.)
x_test=keras.preprocessing.sequence.pad_sequences(x_test, maxlen=None, dtype='int32', padding='post', truncating='pre', value=0.)


In [40]:
x_train.shape

(14016, 51)

In [509]:
print("train size",x_train.shape)
print("test size",x_test.shape)

train size (14016, 3564)
test size (3505, 3564)


In [510]:
y_train = inputLabel_train
y_train = np.asarray(y_train).reshape(-1,1)
y_test = inputLabel_test
y_test = np.asarray(y_test).reshape(-1,1)

In [511]:
# y_train, y_test =  train_test_split(y_train, test_size=0.2,random_state=1)

In [512]:
print("train size",len(y_train))
print("test size",len(y_test))

train size 14016
test size 3505


## Create Model

In [513]:
def create_model():    
    input1 = Input(shape=(x_train.shape[1],))
    x = Dense(1024, activation='relu')(input1)
    x = Dropout(0.5)(x)
    x = BatchNormalization(momentum=0.99, epsilon=0.001)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = BatchNormalization(momentum=0.99, epsilon=0.001)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = BatchNormalization(momentum=0.99, epsilon=0.001)(x)
    x = Dense(16, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = BatchNormalization(momentum=0.99, epsilon=0.001)(x)
    x = Dense(1,activation='linear')(x)
    model = Model(inputs=input1, outputs=x)
    adam  = Adam(lr=0.0001)
    model.compile(optimizer=adam,  loss='mean_absolute_percentage_error')
    
    return model


In [514]:
model = create_model()
model.summary()

Model: "model_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_31 (InputLayer)        (None, 3564)              0         
_________________________________________________________________
dense_149 (Dense)            (None, 1024)              3650560   
_________________________________________________________________
dropout_113 (Dropout)        (None, 1024)              0         
_________________________________________________________________
batch_normalization_113 (Bat (None, 1024)              4096      
_________________________________________________________________
dense_150 (Dense)            (None, 256)               262400    
_________________________________________________________________
dropout_114 (Dropout)        (None, 256)               0         
_________________________________________________________________
batch_normalization_114 (Bat (None, 256)               102

In [515]:
weight_path='./model_baseline_test3.h5'
callbacks_list = [
        ModelCheckpoint(
            weight_path,
            save_best_only=True,
            save_weights_only=True,
            monitor='val_loss',
            mode='min',
            verbose=1
        )
]

In [516]:
model.fit(x_train, y_train, batch_size=32 ,epochs=50, verbose=1, validation_split=0.2,callbacks=callbacks_list)

Train on 11212 samples, validate on 2804 samples
Epoch 1/50
11212/11212 [==============================] - 12s 1ms/step - loss: 491.3832 - val_loss: 231.5566

Epoch 00001: val_loss improved from inf to 231.55657, saving model to ./model_baseline_test3.h5
Epoch 2/50
11212/11212 [==============================] - 7s 601us/step - loss: 402.9977 - val_loss: 194.0217

Epoch 00002: val_loss improved from 231.55657 to 194.02172, saving model to ./model_baseline_test3.h5
Epoch 3/50
11212/11212 [==============================] - 7s 592us/step - loss: 347.9180 - val_loss: 157.2336

Epoch 00003: val_loss improved from 194.02172 to 157.23362, saving model to ./model_baseline_test3.h5
Epoch 4/50
11212/11212 [==============================] - 7s 588us/step - loss: 302.4409 - val_loss: 132.8703

Epoch 00004: val_loss improved from 157.23362 to 132.87033, saving model to ./model_baseline_test3.h5
Epoch 5/50
11212/11212 [==============================] - 7s 591us/step - loss: 264.7050 - val_loss: 118.1

11212/11212 [==============================] - 7s 582us/step - loss: 33.2876 - val_loss: 50.6168

Epoch 00044: val_loss did not improve from 46.71174
Epoch 45/50
11212/11212 [==============================] - 7s 587us/step - loss: 32.5831 - val_loss: 51.2388

Epoch 00045: val_loss did not improve from 46.71174
Epoch 46/50
11212/11212 [==============================] - 7s 592us/step - loss: 32.2355 - val_loss: 51.8013

Epoch 00046: val_loss did not improve from 46.71174
Epoch 47/50
11212/11212 [==============================] - 7s 589us/step - loss: 31.7718 - val_loss: 52.2610

Epoch 00047: val_loss did not improve from 46.71174
Epoch 48/50
11212/11212 [==============================] - 7s 586us/step - loss: 31.1786 - val_loss: 53.1098

Epoch 00048: val_loss did not improve from 46.71174
Epoch 49/50
11212/11212 [==============================] - 7s 586us/step - loss: 30.4485 - val_loss: 53.2708

Epoch 00049: val_loss did not improve from 46.71174
Epoch 50/50
11212/11212 [===============

In [524]:
weight_model='./model_baseline_test1.h5'
model = create_model()
model.load_weights(weight_model)
# model._make_predict_function()
model.summary()

Model: "model_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_33 (InputLayer)        (None, 3519)              0         
_________________________________________________________________
dense_159 (Dense)            (None, 1024)              3604480   
_________________________________________________________________
dropout_121 (Dropout)        (None, 1024)              0         
_________________________________________________________________
batch_normalization_121 (Bat (None, 1024)              4096      
_________________________________________________________________
dense_160 (Dense)            (None, 256)               262400    
_________________________________________________________________
dropout_122 (Dropout)        (None, 256)               0         
_________________________________________________________________
batch_normalization_122 (Bat (None, 256)               102

## Evaluate Model

In [525]:
y_pred = model.predict(x_test)

In [526]:
y_pred2 = y_pred.clip(min=0)

In [527]:
def evaluate(preds, testY):
    diff = preds.flatten() - testY
    percentDiff = (diff / testY) * 100
    absPercentDiff = np.abs(percentDiff)
    mean = np.mean(absPercentDiff)
    std = np.std(absPercentDiff)
    print(mean,std)

In [474]:
evaluate(y_pred,y_test)

46.91694190528806 26.288253607643497


In [529]:
for i in range(50):
    print(y_test[i],y_pred[i])
    print(y_test[i]*MAXVIEW,y_pred[i]*MAXVIEW)

[0.07608696] [0.11794312]
[266.] [412.32916]
[0.24056064] [0.11311649]
[841.] [395.45523]
[0.82122426] [0.1304482]
[2871.] [456.04694]
[0.08352403] [0.12217142]
[292.] [427.11127]
[0.75257437] [0.12687275]
[2631.] [443.54712]
[0.12156751] [0.142565]
[425.] [498.40723]
[0.14273455] [0.12378053]
[499.] [432.73676]
[0.20080092] [0.11713792]
[702.] [409.5142]
[0.10983982] [0.12041637]
[384.] [420.97562]
[0.83066362] [0.142016]
[2904.] [496.4879]
[0.75028604] [0.11423999]
[2623.] [399.383]
[0.29776888] [0.13957931]
[1041.] [487.96927]
[0.09868421] [0.11690778]
[345.] [408.7096]
[0.2271167] [0.13127458]
[794.] [458.93594]
[0.57494279] [0.1223107]
[2010.] [427.5982]
[0.18363844] [0.1194088]
[642.] [417.45316]
[0.21424485] [0.12360439]
[749.] [432.12094]
[0.26058352] [0.11448178]
[911.] [400.2283]
[0.61441648] [0.12002429]
[2148.] [419.60492]
[0.12757437] [0.18252626]
[446.] [638.1118]
[0.73398169] [0.11766489]
[2566.] [411.35645]
[0.31207094] [0.12444988]
[1091.] [435.07678]
[0.46996568] [0.1